In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from torchinfo import summary
from torchvision import datasets, transforms, models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


## Implementatation

In [4]:
class BottleNeckBlock(nn.Module):
    def __init__(self, input_channels, inner_channels, stride=1, projection=None):
        super().__init__()
        self.residual_block = nn.Sequential(
            nn.Conv2d(input_channels, inner_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(inner_channels),
            nn.ReLU(),
            nn.Conv2d(inner_channels, inner_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(inner_channels),
            nn.ReLU(),
            nn.Conv2d(inner_channels, inner_channels*4, kernel_size=1, bias=False),
            nn.BatchNorm2d(inner_channels*4),
        )
        self.projection = projection
        self.relu = nn.ReLU()
    
    def forward(self, x):
        residual = self.residual_block(x)
        shortcut = self.projection(x) if self.projection else x
        out = self.relu(residual + shortcut)
        return out


class ResNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(n_channels, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layers1 = self.make_layers(64, 64, 3, 1)
        self.layers2 = self.make_layers(256, 128, 4, 2)
        self.layers3 = self.make_layers(512, 256, 6, 2)
        self.layers4 = self.make_layers(1024, 512, 3, 2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048, n_classes)
        )
    
    def make_layers(self, input_channels, inner_channels, n_blocks, stride):
        projection = nn.Sequential(
            nn.Conv2d(input_channels, inner_channels*4, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(inner_channels*4)
        )
        layers = [BottleNeckBlock(input_channels, inner_channels, stride, projection)]
        for layer in range(n_blocks-1):
            layers.append(BottleNeckBlock(inner_channels*4, inner_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layers1(x)
        x = self.layers2(x)
        x = self.layers3(x)
        x = self.layers4(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

In [5]:
resnet_model = ResNet(3, 1000)
summary(resnet_model, input_size=(1, 3, 224, 224), col_names=['input_size', 'output_size', 'mult_adds'], depth=4, device='cpu')

Layer (type:depth-idx)                   Input Shape               Output Shape              Mult-Adds
ResNet                                   [1, 3, 224, 224]          [1, 1000]                 --
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 64, 112, 112]         --
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         118,013,952
│    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         [1, 64, 112, 112]         128
│    └─ReLU: 2-3                         [1, 64, 112, 112]         [1, 64, 112, 112]         --
├─MaxPool2d: 1-2                         [1, 64, 112, 112]         [1, 64, 56, 56]           --
├─Sequential: 1-3                        [1, 64, 56, 56]           [1, 256, 56, 56]          --
│    └─BottleNeckBlock: 2-4              [1, 64, 56, 56]           [1, 256, 56, 56]          --
│    │    └─Sequential: 3-1              [1, 64, 56, 56]           [1, 256, 56, 56]          --
│    │    │    └─Conv2d

In [6]:
resnet_torch_model = models.resnet50()
summary(resnet_torch_model, input_size=(1, 3, 224, 224), col_names=['input_size', 'output_size', 'mult_adds'], depth=4, device='cpu')

Layer (type:depth-idx)                   Input Shape               Output Shape              Mult-Adds
ResNet                                   [1, 3, 224, 224]          [1, 1000]                 --
├─Conv2d: 1-1                            [1, 3, 224, 224]          [1, 64, 112, 112]         118,013,952
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 112, 112]         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 64, 56, 56]           [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           [1, 64, 56, 56]           12,845,056
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           [1, 64, 56, 56]           128
│    │    └─Re

## Training

In [7]:
from pathlib import Path

TRAIN_RATIO = 0.8
data_dir = Path('./data/')

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

train_ds = datasets.CIFAR100(data_dir, train=True, download=True, transform=transform)
train_ds, val_ds = random_split(train_ds, (TRAIN_RATIO, 1 - TRAIN_RATIO))
val_ds.transform = transform
test_ds = datasets.CIFAR100(data_dir, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
import wandb
from src.engine import *

config = dict(batch_size=64, lr=5e-4, epochs=20, dataset='CIFAR100')
with wandb.init(project='pytorch-study', name='ResNet50', config=config) as run:
    w_config = run.config
    train_dl = DataLoader(train_ds, batch_size=w_config.batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=w_config.batch_size, shuffle=True)
    
    n_classes = len(train_ds.dataset.classes)
    resnet_mdoel = ResNet(3, n_classes).to(DEVICE)
        
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet_mdoel.parameters(), lr=w_config.lr)
    
    loss_history, acc_history = train(resnet_mdoel, train_dl, val_dl, criterion, optimizer, w_config.epochs, DEVICE, run) 

Epoch=20: 100%|██████████| 20/20 [1:45:02<00:00, 315.15s/it, train_loss=0.225, train_acc=92.48%, val_loss=2.613, val_acc=52.08%]
